### Load libraries and external data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import colored
from colored import stylize
from data import dataloader
import datetime
import json
from neuralnet import nn
import numpy as np
import pandas as pd
import requests
from sklearn.metrics import confusion_matrix
#import pixiedust_node #v≥0.2.5

#### Load data from Firebase.
Requires [Firebase service account credentials](https://console.firebase.google.com/project/tingle-pilot-collected-data/settings/serviceaccounts/adminsdk) in JSON format saved in `./firebase-credentials`.

In [3]:
notepath = "data/notes.csv"
datapath = "data/pilot_data.csv"
corrections_path = "data/corrections.json"

if(
    os.path.exists(notepath) and
    os.path.exists(datapath)
):
    notes = pd.read_csv(notepath)
    pilot_data = pd.read_csv(datapath)
else:
    pilot_data, notes = dataloader.load_from_firebase(
        notes=True,
        start=datetime.datetime(2018,3,7),
        combine=True,
        marked=False
    )
    pilot_data.to_csv(
        datapath,
        index=False
    )
    notes.to_csv(
        notepath,
        index=False
    )

if os.path.exists(corrections_path):
    with open("data/corrections.json", "r") as c:
        corrections = json.load(c)
else:
    corrections = {}

In [10]:
corrections

{'12': {'hand': {'R': {'value': 'L'}}},
 '15': {'step': {'30': {'column': 'step_c2'}}}}

In [4]:
pilot_data = dataloader.index_participants(
    pilot_data
)

In [22]:
pilot_data = dataloader.correct_targets(
    pilot_data,
    'http://matter.childmind.org/js/tinglePilotAppScript.json'
)

In [23]:
pd.options.display.max_columns = 100
dataloader.count_ontarget_samples(pilot_data, True)

ontarget                                           \
participant                      1   2   3   4   5   6   8   9   10  11   12   
step target                                                                    
1    food                         8  15  12   9  18  12  11  11  18  19   16   
2    food                         8   9  11  11  12  13  13  12  21   8   14   
3    food                         5  11  14  11   9  11  15  19  21  23   19   
4    thumb                       13  11  10  12   8  12  10  10  10  19    9   
5    thumb                        9  11  11  12  11  13  12  12   8   7   17   
6    thumb                       11  11  11  12  12  13  12  11  20  17   16   
7    nails                        9  10  16  10  13  12  11  11  16  13   17   
8    nails                       10  12  11  12  13  11  12  13  23  12   12   
9    nails                        9  15  11  11  13  12  12  13  16  15   17   
10   smoke                       10  13  11  15  11  14  11  13   9  17   15   
11   smoke                       10  14  10  13  11  10  11  12  10  13   16   
12   smoke                       11  14  13  13  11  13  11  16   5  16   16   
13   rotate-mouth                75  73  73  69  77  78  74  74  74  64   67   
14   nose                         8  10  10  10   8   8  10  10  19  19   16   
15   nose                        11  10   9  10  12  10  13  10  13  17   14   
16   nose                         9  13  10   7  11  11  11  11  18  16   13   
17   rotate-nose                 73  69  74  75  77  76  77  73  79  66   64   
18   cheek                       10  10   9  12  10   9  10   8  18  20   16   
19   cheek                       10  12  10  12  11  12  10  13  15  12   16   
20   cheek                        9  11  12  12  12  11   9  15  12  10   16   
21   rotate-cheek                76  73  74  76  75  74  76  76  73  64   67   
22   eyebrow                      8  10   8  11  11  11  11  10  18  12   19   
23   eyebrow                     12  11   8  12  12  11  12  11  17  13   13   
24   eyebrow                      9  12   8  13  13  12  12  11  14  13   15   
25   rotate-eyebrow              71  67  75  76  75  75  80  77  76  66   65   
26   top-head                     5  10  10   9   9  11  10   8  14  15   12   
27   top-head                    10  12   9  10  12  11  11   9  16  16   20   
28   top-head                    11  11   9  10  12  13  12  12  17  24   11   
29   rotate-top-head             74  67  72  68  72  73  69  76  76  66   61   
30   back-head                    7  10   7  11  11  12  11   7  19  15   15   
31   back-head                   10  12  12  13  12  12  11  13  14  23   15   
32   back-head                   11  11  10  12  12  12   8  13  14  18   10   
33   rotate-back-head            67  75  72  72  72  76  73  72  77  57   68   
34   rotate-chin                 72  75  64  77  73  76  74  74  79  66   66   
35   rotate-above-ear            74  75  51  73  72  75  83  76  89  64   63   
36   rotate-opposite-cheek       75  74  73  77  73  75  71  71  79  68   64   
37   paint-front-neck            74  66  64  71  71  76  74  74  78  71   65   
38   paint-back-neck             72  70  67  76  70  76  75  67  77  68   66   
39   paint-ear                   72  67  70  76  72  77  79  73  74  69   64   
40   paint-opposite-cheek        75  72  68  77  76  75  73  75  75  65   67   
41   paint-top-head              73  50  49  74  74  74  73  73  77  67  135   
42   paint-front-body            25  26  23  18  11  25  14  17   7  13   19   
43   offbody-ceiling             15  13  15   8  14  16  15  14   6  21   22   
44   offbody-floor               17  14  11  12  13  14  14  17  11   7    6   
45   offbody-+                   30  33  43  26  22  41  20  38  30  49   27   
46   offbody-X                   29  32  38  28  18  42  23  27  18  39   17   
47   offbody-spiral              47  39  63  39  23  64  20  22  40  65   21   

                          

In [24]:
pilot_data = pilot_data[
    (pilot_data.timestamp < 1520451888125)
    |
    (pilot_data.timestamp > 1520451895926)
]

In [25]:
dataloader.count_ontarget_samples(pilot_data, True)

ontarget                                           \
participant                      1   2   3   4   5   6   8   9   10  11   12   
step target                                                                    
1    food                         8  15  12   9  18  12  11  11  18  19   16   
2    food                         8   9  11  11  12  13  13  12  21   8   14   
3    food                         5  11  14  11   9  11  15  19  21  23   19   
4    thumb                       13  11  10  12   8  12  10  10  10  19    9   
5    thumb                        9  11  11  12  11  13  12  12   8   7   17   
6    thumb                       11  11  11  12  12  13  12  11  20  17   16   
7    nails                        9  10  16  10  13  12  11  11  16  13   17   
8    nails                       10  12  11  12  13  11  12  13  23  12   12   
9    nails                        9  15  11  11  13  12  12  13  16  15   17   
10   smoke                       10  13  11  15  11  14  11  13   9  17   15   
11   smoke                       10  14  10  13  11  10  11  12  10  13   16   
12   smoke                       11  14  13  13  11  13  11  16   5  16   16   
13   rotate-mouth                75  73  73  69  77  78  74  74  74  64   67   
14   nose                         8  10  10  10   8   8  10  10  19  19   16   
15   nose                        11  10   9  10  12  10  13  10  13  17   14   
16   nose                         9  13  10   7  11  11  11  11  18  16   13   
17   rotate-nose                 73  69  74  75  77  76  77  73  79  66   64   
18   cheek                       10  10   9  12  10   9  10   8  18  20   16   
19   cheek                       10  12  10  12  11  12  10  13  15  12   16   
20   cheek                        9  11  12  12  12  11   9  15  12  10   16   
21   rotate-cheek                76  73  74  76  75  74  76  76  73  64   67   
22   eyebrow                      8  10   8  11  11  11  11  10  18  12   19   
23   eyebrow                     12  11   8  12  12  11  12  11  17  13   13   
24   eyebrow                      9  12   8  13  13  12  12  11  14  13   15   
25   rotate-eyebrow              71  67  75  76  75  75  80  77  76  66   65   
26   top-head                     5  10  10   9   9  11  10   8  14  15   12   
27   top-head                    10  12   9  10  12  11  11   9  16  16   20   
28   top-head                    11  11   9  10  12  13  12  12  17  24   11   
29   rotate-top-head             74  67  72  68  72  73  69  76  76  66   61   
30   back-head                    7  10   7  11  11  12  11   7  19  15   15   
31   back-head                   10  12  12  13  12  12  11  13  14  23   15   
32   back-head                   11  11  10  12  12  12   8  13  14  18   10   
33   rotate-back-head            67  75  72  72  72  76  73  72  77  57   68   
34   rotate-chin                 72  75  64  77  73  76  74  74  79  66   66   
35   rotate-above-ear            74  75  51  73  72  75  83  76  89  64   63   
36   rotate-opposite-cheek       75  74  73  77  73  75  71  71  79  68   64   
37   paint-front-neck            74  66  64  71  71  76  74  74  78  71   65   
38   paint-back-neck             72  70  67  76  70  76  75  67  77  68   66   
39   paint-ear                   72  67  70  76  72  77  79  73  74  69   64   
40   paint-opposite-cheek        75  72  68  77  76  75  73  75  75  65   67   
41   paint-top-head              73  50  49  74  74  74  73  73  77  67  135   
42   paint-front-body            25  26  23  18  11  25  14  17   7  13   19   
43   offbody-ceiling             15  13  15   8  14  16  15  14   6  21   22   
44   offbody-floor               17  14  11  12  13  14  14  17  11   7    6   
45   offbody-+                   30  33  43  26  22  41  20  38  30  49   27   
46   offbody-X                   29  32  38  28  18  42  23  27  18  39   17   
47   offbody-spiral              47  39  63  39  23  64  20  22  40  65   21   

                          

In [26]:
pilot_data = dataloader.correct_corrections(
    pilot_data,
    corrections
)

In [27]:
dataloader.count_ontarget_samples(pilot_data, True)

ontarget                                           \
participant                      1   2   3   4   5   6   8   9   10  11   12   
step target                                                                    
1    food                         8  15  12   9  18  12  11  11  18  19   16   
2    food                         8   9  11  11  12  13  13  12  21   8   14   
3    food                         5  11  14  11   9  11  15  19  21  23   19   
4    thumb                       13  11  10  12   8  12  10  10  10  19    9   
5    thumb                        9  11  11  12  11  13  12  12   8   7   17   
6    thumb                       11  11  11  12  12  13  12  11  20  17   16   
7    nails                        9  10  16  10  13  12  11  11  16  13   17   
8    nails                       10  12  11  12  13  11  12  13  23  12   12   
9    nails                        9  15  11  11  13  12  12  13  16  15   17   
10   smoke                       10  13  11  15  11  14  11  13   9  17   15   
11   smoke                       10  14  10  13  11  10  11  12  10  13   16   
12   smoke                       11  14  13  13  11  13  11  16   5  16   16   
13   rotate-mouth                75  73  73  69  77  78  74  74  74  64   67   
14   nose                         8  10  10  10   8   8  10  10  19  19   16   
15   nose                        11  10   9  10  12  10  13  10  13  17   14   
16   nose                         9  13  10   7  11  11  11  11  18  16   13   
17   rotate-nose                 73  69  74  75  77  76  77  73  79  66   64   
18   cheek                       10  10   9  12  10   9  10   8  18  20   16   
19   cheek                       10  12  10  12  11  12  10  13  15  12   16   
20   cheek                        9  11  12  12  12  11   9  15  12  10   16   
21   rotate-cheek                76  73  74  76  75  74  76  76  73  64   67   
22   eyebrow                      8  10   8  11  11  11  11  10  18  12   19   
23   eyebrow                     12  11   8  12  12  11  12  11  17  13   13   
24   eyebrow                      9  12   8  13  13  12  12  11  14  13   15   
25   rotate-eyebrow              71  67  75  76  75  75  80  77  76  66   65   
26   top-head                     5  10  10   9   9  11  10   8  14  15   12   
27   top-head                    10  12   9  10  12  11  11   9  16  16   20   
28   top-head                    11  11   9  10  12  13  12  12  17  24   11   
29   rotate-top-head             74  67  72  68  72  73  69  76  76  66   61   
30   back-head                    7  10   7  11  11  12  11   7  19  15   15   
31   back-head                   10  12  12  13  12  12  11  13  14  23   15   
32   back-head                   11  11  10  12  12  12   8  13  14  18   10   
33   rotate-back-head            67  75  72  72  72  76  73  72  77  57   68   
34   rotate-chin                 72  75  64  77  73  76  74  74  79  66   66   
35   rotate-above-ear            74  75  51  73  72  75  83  76  89  64   63   
36   rotate-opposite-cheek       75  74  73  77  73  75  71  71  79  68   64   
37   paint-front-neck            74  66  64  71  71  76  74  74  78  71   65   
38   paint-back-neck             72  70  67  76  70  76  75  67  77  68   66   
39   paint-ear                   72  67  70  76  72  77  79  73  74  69   64   
40   paint-opposite-cheek        75  72  68  77  76  75  73  75  75  65   67   
41   paint-top-head              73  50  49  74  74  74  73  73  77  67  135   
42   paint-front-body            25  26  23  18  11  25  14  17   7  13   19   
43   offbody-ceiling             15  13  15   8  14  16  15  14   6  21   22   
44   offbody-floor               17  14  11  12  13  14  14  17  11   7    6   
45   offbody-+                   30  33  43  26  22  41  20  38  30  49   27   
46   offbody-X                   29  32  38  28  18  42  23  27  18  39   17   
47   offbody-spiral              47  39  63  39  23  64  20  22  40  65   21   

                          

In [28]:
pilot_data = dataloader.update_too_few(
    pilot_data,
    "< 5"
)

In [29]:
dataloader.count_ontarget_samples(pilot_data, True)

ontarget                                           \
participant                      1   2   3   4   5   6   8   9   10  11   12   
step target                                                                    
1    food                         8  15  12   9  18  12  11  11  18  19   16   
2    food                         8   9  11  11  12  13  13  12  21   8   14   
3    food                         5  11  14  11   9  11  15  19  21  23   19   
4    thumb                       13  11  10  12   8  12  10  10  10  19    9   
5    thumb                        9  11  11  12  11  13  12  12   8   7   17   
6    thumb                       11  11  11  12  12  13  12  11  20  17   16   
7    nails                        9  10  16  10  13  12  11  11  16  13   17   
8    nails                       10  12  11  12  13  11  12  13  23  12   12   
9    nails                        9  15  11  11  13  12  12  13  16  15   17   
10   smoke                       10  13  11  15  11  14  11  13   9  17   15   
11   smoke                       10  14  10  13  11  10  11  12  10  13   16   
12   smoke                       11  14  13  13  11  13  11  16   5  16   16   
13   rotate-mouth                75  73  73  69  77  78  74  74  74  64   67   
14   nose                         8  10  10  10   8   8  10  10  19  19   16   
15   nose                        11  10   9  10  12  10  13  10  13  17   14   
16   nose                         9  13  10   7  11  11  11  11  18  16   13   
17   rotate-nose                 73  69  74  75  77  76  77  73  79  66   64   
18   cheek                       10  10   9  12  10   9  10   8  18  20   16   
19   cheek                       10  12  10  12  11  12  10  13  15  12   16   
20   cheek                        9  11  12  12  12  11   9  15  12  10   16   
21   rotate-cheek                76  73  74  76  75  74  76  76  73  64   67   
22   eyebrow                      8  10   8  11  11  11  11  10  18  12   19   
23   eyebrow                     12  11   8  12  12  11  12  11  17  13   13   
24   eyebrow                      9  12   8  13  13  12  12  11  14  13   15   
25   rotate-eyebrow              71  67  75  76  75  75  80  77  76  66   65   
26   top-head                     5  10  10   9   9  11  10   8  14  15   12   
27   top-head                    10  12   9  10  12  11  11   9  16  16   20   
28   top-head                    11  11   9  10  12  13  12  12  17  24   11   
29   rotate-top-head             74  67  72  68  72  73  69  76  76  66   61   
30   back-head                    7  10   7  11  11  12  11   7  19  15   15   
31   back-head                   10  12  12  13  12  12  11  13  14  23   15   
32   back-head                   11  11  10  12  12  12   8  13  14  18   10   
33   rotate-back-head            67  75  72  72  72  76  73  72  77  57   68   
34   rotate-chin                 72  75  64  77  73  76  74  74  79  66   66   
35   rotate-above-ear            74  75  51  73  72  75  83  76  89  64   63   
36   rotate-opposite-cheek       75  74  73  77  73  75  71  71  79  68   64   
37   paint-front-neck            74  66  64  71  71  76  74  74  78  71   65   
38   paint-back-neck             72  70  67  76  70  76  75  67  77  68   66   
39   paint-ear                   72  67  70  76  72  77  79  73  74  69   64   
40   paint-opposite-cheek        75  72  68  77  76  75  73  75  75  65   67   
41   paint-top-head              73  50  49  74  74  74  73  73  77  67  135   
42   paint-front-body            25  26  23  18  11  25  14  17   7  13   19   
43   offbody-ceiling             15  13  15   8  14  16  15  14   6  21   22   
44   offbody-floor               17  14  11  12  13  14  14  17  11   7    6   
45   offbody-+                   30  33  43  26  22  41  20  38  30  49   27   
46   offbody-X                   29  32  38  28  18  42  23  27  18  39   17   
47   offbody-spiral              47  39  63  39  23  64  20  22  40  65   21   

                          

In [30]:
pilot_data[(pilot_data.participant == 15) & (pilot_data.step==4) & (pilot_data.target == "food")]

,distance,hand,pitch,question,roll,section,step,target,thermopile1,thermopile2,thermopile3,thermopile4,timestamp,firstCoordinator,coordinator1,secondCoordinator,step_c2,coordinator2,ontarget,human-readable timestamp,notes,username,participant


In [ ]:
data_points = {}
for i, r in pilot_data_corrected.iterrows():
    if r.participant not in data_points:
        data_points[
            r.participant
        ] = {
            k: {} for k in list(
                range(
                    1,
                    48
                )
            )
        }
    if r.coordinator1 == True and r.coordinator2 == True:
        data_points[
            r.participant
        ][
            r.step
        ][2] = data_points[
            r.participant
        ][
            r.step
        ].get(
            2,
            0
        ) + 1 
    elif r.coordinator1 == True or r.coordinator2 == True:
        data_points[
            r.participant
        ][
            r.step
        ][1] = data_points[
            r.participant
        ][
            r.step
        ].get(
            1,
            0
        ) + 1

In [ ]:
for s in data_points:
    print(str(s))
    for t in data_points[s]:
        ttl = sum(
            data_points[
                s
            ][
                t
            ].values(
            )
        ) if t in data_points[
            s
        ] else str(
            0
        )
        t2 = str(t)
        ttl2 = str(ttl)
        print("{0}: {1}".format(
            t2,
            ttl2
        ), end="\t")
    print("\n")

#### Load [Synaptic](http://caza.la/synaptic/)
If "Error: Cannot find module 'synaptic'", create and run these two cells:

1. ```
cd neuralnet
```

2. ```sh
!npm init -y
!npm install -s synaptic
```

In [ ]:
%%node
var lstm = require('../../tingle-pilot-study/neuralnet/lstm.js');

---
### See all targets, number of available samples and iteration blocks

In [ ]:
with open("neuralnet/targets.json", 'r') as fp:
    targets = json.load(fp)

In [ ]:
pd.options.display.max_columns = 100 # unabridged
dataloader.count_ontarget_samples(pilot_data_corrected, True)

{target: training, [all offtarget], [all onbody offtarget], [all nontraining rotation], [all offbody]}

In [ ]:
for target in list(pilot_data.target.unique()):
    ib = set()
    ib.update(
        list(
            pilot_data.loc[
                pilot_data.target==target
            ]["step"].dropna()
        )
    )
    
    print(": ".join([
        target,
        "{0} on-target samples in {2} {1}".format(
            str(len(pilot_data.loc[
                (pilot_data.target == target) &
                (pilot_data.ontarget)
            ])),
            ", ".join(
                ["%.0f" % step for step in ib],
            ),
            "step" if len(ib) == 1 else "steps"
        )
    ]))

---
### Extract training and testing data
Define targets of interest and corresponding offtargets

with open(
    'data/targets.json',
    'r'
) as fp:
    targets = json.load(
        fp
    )[0]

Set parameters for nn:

In [ ]:
input_signals = [
    "distance",
    "thermopile1",
    "thermopile2",
    "thermopile3",
    "thermopile4"
]
n_samples = [300, 250, 200, 150, 100, 50]
steps = list(range(1, 48))

Get training inputs and outputs, inputs that should evaluate ~true and inputs that should evaluate ~false

In [ ]:
targ = "eyebrow"
data = nn.define_trainer_data(
    pilot_data,
    {
        "target": [targets[targ]],
        "offtarget": targets[targ][1]
    },
    input_signals,
    steps,
    n_samples=n_samples[0]
)

Preview all inputs and training outputs

In [ ]:
data

**Note**: These data take some time to copy across environments. Give the notebook some time between running cells across Python and JavaScript.

---
### Train and test

In [ ]:
%%node
var networks = {};
for n_sample in n_samples:
    for (var target in data) {
      networks[target] = lstm.train_lstm([5,5,2,1], data[target]["train"], 0.06, 0.06, 3000);
    }

In [ ]:
%%node
var test_outputs = {};
for (var target in data) {
    test_outputs[target] = {"true":[],"false":[]};
    for(var iteration=0; iteration < data[target]["test_true"].length; iteration++){
      test_outputs[target]["true"].push(networks[target].activate(data[target]["test_true"][iteration]));
      }
    for(var iteration=0; iteration < data[target]["test_false"].length; iteration++){
      test_outputs[target]["false"].push(networks[target].activate(data[target]["test_false"][iteration]));
    }
}

In [ ]:
test_outputs

---
### See outputs

In [ ]:
def calc_confusion(negative, positive):
    """
    Function to calculate a confusion matrix
    
    Parameters
    ----------
    negative: list of floats
        outputs of neural nets with true negative inputs
        
    positive: list of floats
        outputs of neural nets with true positive inputs
        
    Returns
    -------
    confusion: matrix of floats
        tn, fp, fn, tp
        see http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    """
    ytrue = [
        *[
            0 for output in negative
        ],
        *[
            1 for output in positive
        ]
    ]
    ypredicted = [
        *[
            int(round(o)) for o in negative
        ],
        *[
            int(round(o)) for o in positive
        ]
    ]
    return(confusion_matrix(ytrue, ypredicted))

If the training is adequate x ≈ 0 ∀ x in the following:

In [ ]:
f = {target: [
        outputs for outputs in test_outputs[target]['false']
] for target in targets}
f

If the training is adequate x ≈ 1 ∀ x in the following:

In [ ]:
t = {target: [
        outputs for outputs in test_outputs[target]['true']
] for target in targets}
t

Finally, if training is adequate, f ≪ t:

In [ ]:
f_mean = {
    target: np.mean(f[target]) for target in targets if target in f and len(f[target])
}
t_mean = {
    target: np.mean(t[target]) for target in targets if target in t and len(t[target])
}
for target in t_mean:
    print(target)
    print(
        "f = {0}\nt = {1}\n{0} ≪ {1} ?\n".format(
            str(f_mean[target]),
            str(t_mean[target])
        ) if f_mean[target] < t_mean[target] else "f = {0}\nt = {1}\n{2}".format(
            str(f_mean[target]),
            str(t_mean[target]),
            stylize(
                "Nope. f > t\n",
                colored.fg("red")
            )
        )
    )
for target in t_mean:
    print("{0}: f = {1:.4f} < t = {2:.4f}".format(
        target,
        f_mean[target],
        t_mean[target]
    ))